In [3]:
import nvidia_tlt_inference as inference

model_path = "/models/resnet18_trafficcamnet.tlt"
image_path = "images/image"

inference.infer(tlt_model_path,image_path)

ModuleNotFoundError: No module named 'tlt.models'